In [ ]:
import os
import random 
import numpy as np
import pandas as pd
# from scipy.signal import resample
from glob import glob 
import scipy.io 


In [ ]:
class PAMAP2Reader(object):
    def __init__(self, root_path):
        self.root_path = root_path
        self.readPamap2()

    def readFile(self, file_path, cols):
        all_data = {"data": {}, "target": {}, 'collection': []}
        prev_action = -1
        starting = True
        # action_seq = []
        action_ID = 0

        for l in open(file_path).readlines():
            s = l.strip().split()
            if s[1] != "0":
                if (prev_action != int(s[1])):
                    if not(starting):
                        df = pd.DataFrame(action_seq)
                        intep_df = df.interpolate(method='linear', limit_direction='backward', axis=0)
                        intep_data = intep_df.values 
                        all_data['data'][action_ID] = np.array(intep_data)
                        all_data['target'][action_ID] = prev_action
                        action_ID+=1
                    action_seq = []
                else:
                    starting = False
                intm_data = s[3:]
                data_seq = np.array(intm_data)[cols].astype(np.float16)
                # data_seq[np.isnan(data_seq)] = 0
                action_seq.append(data_seq)
                prev_action = int(s[1])
                # print(prev_action)
                all_data['collection'].append(data_seq)
        else: 
            if len(action_seq) > 1:
                df = pd.DataFrame(action_seq)
                intep_df = df.interpolate(method='linear', limit_direction='backward', axis=0)
                intep_data = intep_df.values
                all_data['data'][action_ID] = np.array(intep_data)
                all_data['target'][action_ID] = prev_action
        return all_data

    def readPamap2Files(self, filelist, cols, labelToId):
        data = []
        labels = []
        collection = []
        for i, filename in enumerate(filelist):
            print('Reading file %d of %d' % (i+1, len(filelist)))
            fpath = os.path.join(self.root_path, filename)
            file_data = self.readFile(fpath, cols)
            data.extend(list(file_data['data'].values()))
            labels.extend(list(file_data['target'].values()))
            collection.extend(file_data['collection'])
        return np.asarray(data), np.asarray(labels, dtype=int), np.array(collection)

    def readPamap2(self):
        files = ['subject101.dat', 'subject102.dat','subject103.dat','subject104.dat', 'subject105.dat', 'subject106.dat', 'subject107.dat', 'subject108.dat', 'subject109.dat', 'subject110.dat', 'subject111.dat', 'subject112.dat', 'subject113.dat', 'subject114.dat']
            
        label_map = [
            #(0, 'other'),
            (1, 'lying'),
            (2, 'sitting'),
            (3, 'standing'),
            (4, 'walking'),
            (5, 'running'),
            (6, 'cycling'),
            (7, 'Nordic walking'),
            (9, 'watching TV'),
            (10, 'computer work'),
            (11, 'car driving'),
            (12, 'ascending stairs'),
            (13, 'descending stairs'),
            (16, 'vacuum cleaning'),
            (17, 'ironing'),
            (18, 'folding laundry'),
            (19, 'house cleaning'),
            (20, 'playing soccer'),
            (24, 'rope jumping')
        ]
        labelToId = {x[0]: i for i, x in enumerate(label_map)}
        # print "label2id=",labelToId
        idToLabel = [x[1] for x in label_map]
        # print "id2label=",idToLabel
        cols = [1,2,3,7,8,9,10,11,12,17,18,19,23,24,25,26,27,28,33,34,35,39,40,41,42,43,44]
        self.cols = cols
        # print "cols",cols
        self.data, self.targets, self.all_data = self.readPamap2Files(files, cols, labelToId)
        # self.data = self.data[:, :, cols]
        # print(self.data)
        # nan_perc = np.isnan(self.data).astype(int).mean()
        # print("null value percentage ", nan_perc)
        # f = lambda x: labelToId[x]
        self.targets = np.array([labelToId[i] for i in list(self.targets)])
        self.label_map = label_map
        self.idToLabel = idToLabel
        # return data, idToLabel

    def resample(self, signal, freq=10):
        step_size = int(100/freq)
        seq_len, _ = signal.shape 
        resample_indx = np.arange(0, seq_len, step_size)
        resampled_sig = signal[resample_indx, :]
        return resampled_sig

    def windowing(self, signal, window_len, overlap):
        seq_len = int(window_len*100) # 100Hz compensation 
        overlap_len = int(overlap*100) # 100Hz
        l, _ = signal.shape
        if l > seq_len:
            windowing_points = np.arange(start=0, stop=l-seq_len, step=seq_len-overlap_len, dtype=int)[:-1]

            windows = [signal[p:p+seq_len, :] for p in windowing_points]
        else:
            windows = []
        return windows

    def resampling(self, data, targets, window_size, window_overlap, resample_freq):
        assert len(data) == len(targets), "# action data & # action labels are not matching"
        all_data, all_ids, all_labels = [], [], []
        for i, d in enumerate(data):
            # print(">>>>>>>>>>>>>>>  ", np.isnan(d).mean())
            label = targets[i]
            windows = self.windowing(d, window_size, window_overlap)
            for w in windows:
                # print(np.isnan(w).mean(), label, i)
                resample_sig = self.resample(w, resample_freq)
                # print(np.isnan(resample_sig).mean(), label, i)
                all_data.append(resample_sig)
                all_ids.append(i+1)
                all_labels.append(label)

        return all_data, all_ids, all_labels

    def generate(self, unseen_classes, window_size=5.21, window_overlap=1, resample_freq=10, smoothing=False, normalize=False, seen_ratio=0.2, unseen_ratio=0.8):
        
        def smooth(x, window_len=11, window='hanning'):
            if x.ndim != 1:
                    raise Exception('smooth only accepts 1 dimension arrays.')
            if x.size < window_len:
                    raise Exception("Input vector needs to be bigger than window size.")
            if window_len<3:
                    return x
            if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
                    raise Exception("Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'")
            s=np.r_[2*x[0]-x[window_len-1::-1],x,2*x[-1]-x[-1:-window_len:-1]]
            if window == 'flat': #moving average
                    w=np.ones(window_len,'d')
            else:  
                    w=eval('np.'+window+'(window_len)')
            y=np.convolve(w/w.sum(),s,mode='same')
            return y[window_len:-window_len+1]

        # assert all([i in list(self.label_map.keys()) for i in unseen_classes]), "Unknown Class label!"
        seen_classes = [i for i in range(len(self.idToLabel)) if i not in unseen_classes]
        unseen_mask = np.in1d(self.targets, unseen_classes)

        # build seen dataset 
        seen_data = self.data[np.invert(unseen_mask)]
        seen_targets = self.targets[np.invert(unseen_mask)]

        # build unseen dataset
        unseen_data = self.data[unseen_mask]
        unseen_targets = self.targets[unseen_mask]

        # resampling seen and unseen datasets 
        seen_data, seen_ids, seen_targets = self.resampling(seen_data, seen_targets, window_size, window_overlap, resample_freq)
        unseen_data, unseen_ids, unseen_targets = self.resampling(unseen_data, unseen_targets, window_size, window_overlap, resample_freq)

        seen_data, seen_targets = np.array(seen_data), np.array(seen_targets)
        unseen_data, unseen_targets = np.array(unseen_data), np.array(unseen_targets)

        if normalize:
            a, b, nft = seen_data.shape 
            intm_sdata = seen_data.reshape((-1, nft))
            intm_udata = unseen_data.reshape((-1, nft))

            scaler = MinMaxScaler()
            norm_sdata = scaler.fit_transform(intm_sdata)
            norm_udata = scaler.transform(intm_udata)

            seen_data = norm_sdata.reshape(seen_data.shape)
            unseen_data = norm_udata.reshape(unseen_data.shape)

        if smoothing:
            seen_data = np.apply_along_axis(smooth, axis=1, arr=seen_data)
            unseen_data = np.apply_along_axis(smooth, axis=1, arr=unseen_data)
        # train-val split
        seen_index = list(range(len(seen_targets)))
        random.shuffle(seen_index)
        split_point = int((1-seen_ratio)*len(seen_index))
        fst_index, sec_index = seen_index[:split_point], seen_index[split_point:]
        # print(type(fst_index), type(sec_index), type(seen_data), type(seen_targets))
        X_seen_train, X_seen_val, y_seen_train, y_seen_val = seen_data[fst_index,:], seen_data[sec_index,:], seen_targets[fst_index], seen_targets[sec_index]
        

        data = {'train': {
                        'X': X_seen_train,
                        'y': y_seen_train
                        },
                'eval-seen':{
                        'X': X_seen_val,
                        'y': y_seen_val
                        },
                'test': {
                        'X': unseen_data,
                        'y': unseen_targets
                        },
                'seen_classes': seen_classes,
                'unseen_classes': unseen_classes
                }

        return data

In [1]:
import numpy as np 
import pandas as pd 

In [2]:
sample = np.random.rand(100, 21)
sample.shape

(100, 21)

In [4]:
sample.min(axis=0).shape

(21,)

In [5]:
feat_max = sample.max(axis=0)
feat_min = sample.min(axis=0)
# min-max normalized feats. 
norm_feat = (sample-feat_min)/(feat_max-feat_min)
norm_feat.shape

(100, 21)

In [7]:
norm_feat.max()

1.0

In [4]:
resamples = sample.reshape(5, -1, 21)
resamples.shape

(5, 20, 21)

In [5]:
resample_means = resamples.mean(axis=0)
resample_means.shape

(20, 21)

In [6]:
resample_stds = resamples.std(axis=0)
resample_stds.shape

(20, 21)

In [9]:
full_resample = np.vstack([resample_means, resample_stds]).reshape((-1,),order='F')
full_resample.shape

(840,)

In [10]:
class PAMAP2ReaderV2(object):
    def __init__(self, root_path):
        self.root_path = root_path
        self.readPamap2()

    def readFile(self, file_path, cols):
        all_data = {"data": {}, "target": {}, 'collection': []}
        prev_action = -1
        starting = True
        # action_seq = []
        action_ID = 0

        for l in open(file_path).readlines():
            s = l.strip().split()
            if s[1] != "0":
                if (prev_action != int(s[1])):
                    if not(starting):
                        df = pd.DataFrame(action_seq)
                        intep_df = df.interpolate(method='linear', limit_direction='backward', axis=0)
                        intep_data = intep_df.values 
                        all_data['data'][action_ID] = np.array(intep_data)
                        all_data['target'][action_ID] = prev_action
                        action_ID+=1
                    action_seq = []
                else:
                    starting = False
                intm_data = s[3:]
                data_seq = np.array(intm_data)[cols].astype(np.float16)
                # data_seq[np.isnan(data_seq)] = 0
                action_seq.append(data_seq)
                prev_action = int(s[1])
                # print(prev_action)
                all_data['collection'].append(data_seq)
        else: 
            if len(action_seq) > 1:
                df = pd.DataFrame(action_seq)
                intep_df = df.interpolate(method='linear', limit_direction='backward', axis=0)
                intep_data = intep_df.values
                all_data['data'][action_ID] = np.array(intep_data)
                all_data['target'][action_ID] = prev_action
        return all_data

    def readPamap2Files(self, filelist, cols, labelToId):
        data = []
        labels = []
        collection = []
        for i, filename in enumerate(filelist):
            print('Reading file %d of %d' % (i+1, len(filelist)))
            fpath = os.path.join(self.root_path, filename)
            file_data = self.readFile(fpath, cols)
            data.extend(list(file_data['data'].values()))
            labels.extend(list(file_data['target'].values()))
            collection.extend(file_data['collection'])
        return np.asarray(data), np.asarray(labels, dtype=int), np.array(collection)

    def readPamap2(self):
        files = ['subject101.dat', 'subject102.dat','subject103.dat','subject104.dat', 'subject105.dat', 'subject106.dat', 'subject107.dat', 'subject108.dat', 'subject109.dat', 'subject110.dat', 'subject111.dat', 'subject112.dat', 'subject113.dat', 'subject114.dat']
            
        label_map = [
            #(0, 'other'),
            (1, 'lying'),
            (2, 'sitting'),
            (3, 'standing'),
            (4, 'walking'),
            (5, 'running'),
            (6, 'cycling'),
            (7, 'Nordic walking'),
            (9, 'watching TV'),
            (10, 'computer work'),
            (11, 'car driving'),
            (12, 'ascending stairs'),
            (13, 'descending stairs'),
            (16, 'vacuum cleaning'),
            (17, 'ironing'),
            (18, 'folding laundry'),
            (19, 'house cleaning'),
            (20, 'playing soccer'),
            (24, 'rope jumping')
        ]
        labelToId = {x[0]: i for i, x in enumerate(label_map)}
        # print "label2id=",labelToId
        idToLabel = [x[1] for x in label_map]
        # print "id2label=",idToLabel
        cols = [1,2,3,7,8,9,10,11,12,17,18,19,23,24,25,26,27,28,33,34,35,39,40,41,42,43,44]
        self.cols = cols
        # print "cols",cols
        self.data, self.targets, self.all_data = self.readPamap2Files(files, cols, labelToId)
        # self.data = self.data[:, :, cols]
        # print(self.data)
        # nan_perc = np.isnan(self.data).astype(int).mean()
        # print("null value percentage ", nan_perc)
        # f = lambda x: labelToId[x]
        self.targets = np.array([labelToId[i] for i in list(self.targets)])
        self.label_map = label_map
        self.idToLabel = idToLabel
        # return data, idToLabel

    def aggregate(self, signal, width):
        a,b = signal.shape
        resamples = signal.reshape((width, -1, b))
        means = resamples.astype(np.float64).mean(axis=0)
        stds = resamples.astype(np.float64).std(axis=0)
        mergered = np.hstack((means,stds))
        # print(signal.shape, means.shape, stds.shape, mergered.shape)
        return mergered

    def windowing(self, signal, window_len, overlap):
        seq_len = int(window_len*100) # 100Hz compensation 
        overlap_len = int(overlap*100) # 100Hz
        l, _ = signal.shape
        if l > seq_len:
            windowing_points = np.arange(start=0, stop=l-seq_len, step=seq_len-overlap_len, dtype=int)[:-1]

            windows = [signal[p:p+seq_len, :] for p in windowing_points]
        else:
            windows = []
        return windows

    def resampling(self, data, targets, window_size, window_overlap, resample_freq):
        assert len(data) == len(targets), "# action data & # action labels are not matching"
        all_data, all_ids, all_labels = [], [], []
        for i, d in enumerate(data):
            # print(">>>>>>>>>>>>>>>  ", np.isnan(d).mean())
            label = targets[i]
            windows = self.windowing(d, window_size, window_overlap)
            for w in windows:
                # print(np.isnan(w).mean(), label, i)
                resample_sig = self.aggregate(w, resample_freq)
                # print(np.isnan(resample_sig).mean(), label, i)
                all_data.append(resample_sig)
                all_ids.append(i+1)
                all_labels.append(label)

        return all_data, all_ids, all_labels

    def generate(self, unseen_classes, window_size=5.21, window_overlap=1, resample_freq=10, smoothing=False, normalize=False, seen_ratio=0.2, unseen_ratio=0.8):
        
        def smooth(x, window_len=11, window='hanning'):
            if x.ndim != 1:
                    raise Exception('smooth only accepts 1 dimension arrays.')
            if x.size < window_len:
                    raise Exception("Input vector needs to be bigger than window size.")
            if window_len<3:
                    return x
            if not window in ['flat', 'hanning', 'hamming', 'bartlett', 'blackman']:
                    raise Exception("Window is on of 'flat', 'hanning', 'hamming', 'bartlett', 'blackman'")
            s=np.r_[2*x[0]-x[window_len-1::-1],x,2*x[-1]-x[-1:-window_len:-1]]
            if window == 'flat': #moving average
                    w=np.ones(window_len,'d')
            else:  
                    w=eval('np.'+window+'(window_len)')
            y=np.convolve(w/w.sum(),s,mode='same')
            return y[window_len:-window_len+1]

        # assert all([i in list(self.label_map.keys()) for i in unseen_classes]), "Unknown Class label!"
        seen_classes = [i for i in range(len(self.idToLabel)) if i not in unseen_classes]
        unseen_mask = np.in1d(self.targets, unseen_classes)

        # build seen dataset 
        seen_data = self.data[np.invert(unseen_mask)]
        seen_targets = self.targets[np.invert(unseen_mask)]

        # build unseen dataset
        unseen_data = self.data[unseen_mask]
        unseen_targets = self.targets[unseen_mask]

        # resampling seen and unseen datasets 
        seen_data, seen_ids, seen_targets = self.resampling(seen_data, seen_targets, window_size, window_overlap, resample_freq)
        unseen_data, unseen_ids, unseen_targets = self.resampling(unseen_data, unseen_targets, window_size, window_overlap, resample_freq)

        seen_data, seen_targets = np.array(seen_data), np.array(seen_targets)
        unseen_data, unseen_targets = np.array(unseen_data), np.array(unseen_targets)

        if normalize:
            a, b, nft = seen_data.shape 
            intm_sdata = seen_data.reshape((-1, nft))
            intm_udata = unseen_data.reshape((-1, nft))

            scaler = MinMaxScaler()
            norm_sdata = scaler.fit_transform(intm_sdata)
            norm_udata = scaler.transform(intm_udata)

            seen_data = norm_sdata.reshape(seen_data.shape)
            unseen_data = norm_udata.reshape(unseen_data.shape)

        if smoothing:
            seen_data = np.apply_along_axis(smooth, axis=1, arr=seen_data)
            unseen_data = np.apply_along_axis(smooth, axis=1, arr=unseen_data)

        # train-val split
        seen_index = list(range(len(seen_targets)))
        random.shuffle(seen_index)
        split_point = int((1-seen_ratio)*len(seen_index))
        fst_index, sec_index = seen_index[:split_point], seen_index[split_point:]
        # print(type(fst_index), type(sec_index), type(seen_data), type(seen_targets))
        X_seen_train, X_seen_val, y_seen_train, y_seen_val = seen_data[fst_index,:], seen_data[sec_index,:], seen_targets[fst_index], seen_targets[sec_index]
        

        data = {'train': {
                        'X': X_seen_train,
                        'y': y_seen_train
                        },
                'eval-seen':{
                        'X': X_seen_val,
                        'y': y_seen_val
                        },
                'test': {
                        'X': unseen_data,
                        'y': unseen_targets
                        },
                'seen_classes': seen_classes,
                'unseen_classes': unseen_classes
                }

        return data